# Import data

In [1]:
import pandas as pd
import numpy as np

In [25]:
df=pd.read_excel('Data/Active - Wholesale & retail.xlsx', sheetname='Results')

In [26]:
len(df)

33791

# Extract Twitter handle + Website

In [27]:
from pws import Bing
from pws import Google

import re

In [28]:
companies=df['Company name'][1000:1500]

In [29]:
# clean to remove 'limited etc'

companies=companies.apply(lambda x: x.replace('PLC', ''))
companies=companies.apply(lambda x: x.replace('P L C', ''))
companies=companies.apply(lambda x: x.replace('P.L.C.', ''))
companies=companies.apply(lambda x: x.replace('LTD', ''))
companies=companies.apply(lambda x: x.replace('LIMITED', ''))

In [30]:
website_links=[]
website_links_2=[]
twitter_links=[]
twitter_links_2=[]

In [31]:
for x in companies:
    try:   
        twitter_result=Bing.search(x + 'Twitter', num=2, start=0, sleep=True, country_code='uk')
        twitter_links.append(twitter_result['results'][0]['link'])
        twitter_links_2.append(twitter_result['results'][1]['link'])
    except:
        twitter_links.append('error in retrieving twitter')
        twitter_links_2.append('error in retrieving twitter')
        continue

In [32]:
for x in companies:
    try:   
        website_result=Bing.search(x + 'Website', num=2, start=0, sleep=True, country_code='uk')
        website_links.append(website_result['results'][0]['link'])
        website_links_2.append(website_result['results'][1]['link'])
    except:
        website_links.append('error in retrieving website')
        website_links_2.append('error in retrieving website')
        continue

In [33]:
len(twitter_links_2)

501

In [34]:
DF=pd.DataFrame({'company': companies, 'Twitter': twitter_links, 'Twitter 2': twitter_links_2,
                 'Website': website_links,'Website 2': website_links_2})

In [35]:
DF.to_csv('twitter + sites 1000-1500.csv')

# Cleaning Twitter Handles

In [54]:
df=pd.read_csv('twitter + sites 1500-2500.csv')
df=df.drop({'Unnamed: 0'}, 1)
df=df[['company', 'Website', 'Website 2', 'Twitter', 'Twitter 2']]

In [55]:
def selectTwitter(row):
    if type(row)==str:
        if row[8] != 't':
            return np.nan
        else:
            return row
    else:
        return np.nan
df['Twitter']=df['Twitter'].apply(selectTwitter)
df['Twitter 2']=df['Twitter 2'].apply(selectTwitter)

In [56]:
df['Twitter']=df['Twitter'].fillna(df['Twitter 2'])
df=df.drop({'Twitter 2'},1)

In [57]:
def extracthandle(row):
    try:
        if type(row)==str:
            row= row[20:]

            row=re.search(r'\w+', row)
            return row.group()
    except:
        pass

In [58]:
df['Twitter']=df['Twitter'].apply(extracthandle)

In [59]:
df['Twitter'].head()

0        None
1        None
2    GCmotors
3        None
4        None
Name: Twitter, dtype: object



-- 140 twitter handles extracted

# Run Tweet Scraper

In [60]:
import tweepy 
import csv


In [68]:
#Twitter API credentials
#consumer_key = "CJqhuEFhIFfkw1k43htNOnJeX"
#consumer_secret = "Hm46gor4LVvv4N6X7uzYInHBSaLBBCOmlIoqKxj0YWXnevD8c2"
#access_key = "64435602-drySR7XgKqQUvawnV9ZPp5ZqxvpGE3dTAKhmAp3q6"
#access_secret = "F9CoAbW5X3TGv6Z89DK2c84k50GAAFASPuaEcy36dFDaz"

# issieibb
#consumer_key = "XS1yXEmxTiCkXGah3aFyMsMEa"
#consumer_secret = "Wllivye1lKLMreewdW3fEj7liXJBensvsEsofCUy3fFekDlsnk"
#access_key = "754056966448750592-fTFelEPHOS1HE54pTQuubJmjfzXwpt2"
#access_secret = "ctVlg98kB7gl2j7FNsq4xxN28JDngYEiicClYQ1DDVYkH"

In [62]:
users=df['Twitter'].dropna(how='any', axis=0)
users=users.reset_index()
users=users.drop({'index'},1)

In [48]:
def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []  
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print ("getting tweets before %s" % (oldest))
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        print ("...%s tweets downloaded so far" % (len(alltweets)))
    
    #transform the tweepy tweets into a 2D array that will populate the csv 
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8"), screen_name, tweet.retweet_count, tweet.favorite_count] for tweet in alltweets]
    
    #write the csv  
    with open('Tweets/%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text", "User", 'num_RTs', 'num_Favourites'])
        
        writer.writerows(outtweets)
        print('done')
    pass


In [153]:
#users.Twitter

In [71]:
for user in users.Twitter[394:]:
    if __name__ == '__main__':
        try:
    #pass in the username of the account you want to download
            get_all_tweets(user)
        except:
            print ('error at %s ' %user)
            pass

getting tweets before 743895818797748223
...400 tweets downloaded so far
getting tweets before 585411863128756223
...600 tweets downloaded so far
getting tweets before 446041174709522432
...800 tweets downloaded so far
getting tweets before 317006741432831999
...1000 tweets downloaded so far
getting tweets before 95582097611300863
...1015 tweets downloaded so far
getting tweets before 1049310918
...1015 tweets downloaded so far
done
getting tweets before 171389989958074367
...6 tweets downloaded so far
done
getting tweets before 824305863628353535
...400 tweets downloaded so far
getting tweets before 781098282521165823
...600 tweets downloaded so far
getting tweets before 760461450640920575
...800 tweets downloaded so far
getting tweets before 699608814295896067
...1000 tweets downloaded so far
getting tweets before 624631519966052352
...1200 tweets downloaded so far
getting tweets before 573434033440456703
...1400 tweets downloaded so far
getting tweets before 512201816097189888
...16

getting tweets before 812019965280813055
...400 tweets downloaded so far
getting tweets before 766428182958473216
...600 tweets downloaded so far
getting tweets before 725341068208779263
...797 tweets downloaded so far
getting tweets before 672613626604224511
...997 tweets downloaded so far
getting tweets before 549439633852223487
...1195 tweets downloaded so far
getting tweets before 458273769727864831
...1395 tweets downloaded so far
getting tweets before 352820055442079745
...1595 tweets downloaded so far
getting tweets before 188414001393319935
...1793 tweets downloaded so far
getting tweets before 27957166207
...1993 tweets downloaded so far
getting tweets before 3401178212
...2122 tweets downloaded so far
getting tweets before 1289034316
...2122 tweets downloaded so far
done
getting tweets before 849576123562373119
...400 tweets downloaded so far
getting tweets before 814473658769043455
...600 tweets downloaded so far
getting tweets before 762662027722625023
...800 tweets downloa

...1000 tweets downloaded so far
getting tweets before 750386262754942975
...1200 tweets downloaded so far
getting tweets before 715585700146585599
...1400 tweets downloaded so far
getting tweets before 693120181153959937
...1600 tweets downloaded so far
getting tweets before 646370905137000447
...1800 tweets downloaded so far
getting tweets before 603235172062990335
...2000 tweets downloaded so far
getting tweets before 568190735780073474
...2200 tweets downloaded so far
getting tweets before 522066054781341695
...2400 tweets downloaded so far
getting tweets before 474309875036864516
...2600 tweets downloaded so far
getting tweets before 427903817443979263
...2800 tweets downloaded so far
getting tweets before 378207885211471872
...3000 tweets downloaded so far
getting tweets before 341950746129993728
...3200 tweets downloaded so far
getting tweets before 316586095993290751
...3250 tweets downloaded so far
getting tweets before 309770465788964863
...3250 tweets downloaded so far
done


...989 tweets downloaded so far
getting tweets before 705036996469792767
...1188 tweets downloaded so far
getting tweets before 669102373118283775
...1385 tweets downloaded so far
getting tweets before 619843607479123968
...1580 tweets downloaded so far
getting tweets before 600778499218104319
...1778 tweets downloaded so far
getting tweets before 576346663679307775
...1972 tweets downloaded so far
getting tweets before 539810067592523775
...2168 tweets downloaded so far
getting tweets before 515550342084177919
...2360 tweets downloaded so far
getting tweets before 482104581913329664
...2548 tweets downloaded so far
getting tweets before 452146457005813759
...2746 tweets downloaded so far
getting tweets before 433724612674088959
...2943 tweets downloaded so far
getting tweets before 401384936428806143
...3139 tweets downloaded so far
getting tweets before 380326911718998015
...3172 tweets downloaded so far
getting tweets before 377430705925804032
...3172 tweets downloaded so far
done
g

In [70]:
#users[users=='westcoast_uk']
users.iloc[np.where(users=='dbauder')]
#users

,Twitter
394,dbauder


In [116]:
users

,Twitter
0,aspenpharma
1,insidecitywest
2,westovergroup
3,Pontaragrain
4,DraytonGroup
5,Coty
6,Alpha_LSG
7,LauraAshleyUK
8,berrygardensltd
9,sandicliffe
